In [96]:
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

from sklearn.impute import KNNImputer

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report


from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error


from sklearn.metrics import plot_confusion_matrix
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline


%matplotlib inline

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [97]:
train= pd.read_csv("/kaggle/input/titanic/train.csv")
train.info()

In [98]:
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
test_data.info()

In [99]:
test_data1 = pd.read_csv("/kaggle/input/titanic/test.csv")

In [100]:
train['Age']=np.round(train['Age'])
train = train.drop(columns=['Cabin','Ticket', 'Name','PassengerId'])

columns=['Survived','Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']

In [101]:
test_data = test_data.drop(columns=['Cabin','Ticket', 'Name','PassengerId'])

In [102]:
label_encoder = LabelEncoder()


train["Sex"] = label_encoder.fit_transform(train["Sex"])

#test["Embarked"] = label_encoder.fit_transform(test["Embarked"])
#test["Sex"] = label_encoder.fit_transform(test["Sex"])


nmap = {'Q':1, 'S':0, 'C':2} 
smap = {'female':1, 'male':0} 
train['Embarked1'] = train['Embarked'].map(nmap)
train['Sex1'] = train['Sex'].map(smap)


train[train['Embarked']==3]=2


train['Embarked'].value_counts()
train['Sex'].value_counts()

In [103]:
test_data["Sex"] = label_encoder.fit_transform(test_data["Sex"])
test_data['Sex'].value_counts()

In [104]:
test_data['Embarked'] = test_data['Embarked'].map(nmap)
test_data['Embarked'].value_counts()

In [105]:
train['Embarked'] = train['Embarked'].map(nmap)
train['Embarked'].value_counts()

In [106]:
train = train.drop(columns='Embarked1')

In [107]:
imputer = KNNImputer(n_neighbors=5) 
df = imputer.fit_transform(train)
df = pd.DataFrame(df, columns = columns)

In [108]:
columns1 = test_data.columns

imputer1 = KNNImputer(n_neighbors=5) 
df1 = imputer1.fit_transform(test_data)
df1 = pd.DataFrame(df1, columns = columns1)

In [160]:
#als = df[df['Age']<15]

#als

als=df.copy()
als

In [198]:
df = df.sort_values(by='Age')
df.iloc[:156,3]=0
df.iloc[156:677,3]=1
df.iloc[677:,3]=2

In [199]:
df['Age'].value_counts()

In [109]:
df['Age']=np.round(df['Age'])
df['Age'] = pd.to_numeric(df['Age'], downcast='signed') 

In [200]:
df['Age'].value_counts()

In [50]:
df.info()

In [210]:
from sklearn.ensemble import RandomForestClassifier

y = df["Survived"]

features = ["Pclass", "Sex", "Fare", "Parch", 'Age']
X = df[features]
X_test = df1[features]



# Create a k-NN classifier with 6 neighbors: knn
knn = KNeighborsClassifier(n_neighbors=5)
# Fit the classifier to the training data
model = knn.fit(X, y)
predictions = model.predict(X_test)


#X, y = make_classification(n_features=4, random_state=0)
#clf = make_pipeline(StandardScaler(),LinearSVC(random_state=0, tol=1e-5)
#model = clf.fit(X, y)
#train_accuracy = clf.score(X_train, y_train)
#test_accuracy = clf.score(X_test, y_test)


predictions=np.array(predictions,int)

output = pd.DataFrame({'PassengerId': test_data1.PassengerId, 'Survived': predictions})
output.to_csv('my_submission.csv', index=False)
print("Your submission was successfully saved!")

# Selam Ke